In [27]:
import A1_data_prep
import A4_program_allocation_plan
import pandas as pd
from calitp import *

pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [28]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"

In [29]:
allocation = A1_data_prep.clean_allocation()

In [55]:
allocation = A1_data_prep.clean_up_columns(allocation)

In [30]:
project = A1_data_prep.clean_project()

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed


In [31]:
project.head(1)

,project_award_year,project_project_#,project_grant_recipient,project_project_title,project_ppno,project_district,project_county,project_project_description,project_master_agreement_number,project_master_agreement_expiration_date,project_project_manager,project_regional_coordinator,project_technical_assistance_calitp__y_n_,project_technical_assistance_fleet__y_n_,project_technical_assistance_network_integration__y_n_,project_technical_assistance_priority_population__y_n_,project_total_project_cost,project_tircp_award_amount__$_,project_allocated_amount,project_unallocated_amount,project_percentage_allocated,project_expended_amount,project_other_funds_involved,project_award_cycle,project_estimated_tircp_ghg_reductions,project_cost_per_ghg_ton_reduced,project_increased_ridership,project_service_integration,project_improve_safety,project_project_readiness,project_funding_leverage,project_multi_agency_coordination_integration,project_priority_population_benefits___ab_1550_community_benefits,project_housing_co_benefits,project_local_agency_address,project_local_agency_city,project_local_agency_zip,project_local_agency_contact,project_local_agency_email,project_local_agency_phone_number,project_comments_additional_contacts
0,2015,1,Antelope Valley Transit Authority,Regional Transit Interconnectivity & Environmental Sustability,CP005,7,LA,Purchase 13 60-foot articulated BRT buses and 16 45-foot electric commuter buses,64AVTA2015MA,2024-04-01 00:00:00,Yesenia Ochoa,Ryan Greenway,None,None,None,None,39478000,24403000,24403000,0,1.0,21714177.53,NaN,1,"195,380 tons",None,None,None,None,None,None,None,None,None,"42210 6th Street West\nLancaster, CA 93534",Lancaster,93534,Judy Fry,Jfry@avta.com,(611) 729-2234,None


In [57]:
allocation.columns

Index(['Award Year', '#', 'Grant Recipient', 'Implementing Agency', 'Ppno',
       'Id', 'Ea', 'Components', 'Phase', 'Amount', 'Expended Amount',
       'Sb1 Funding', 'Sb1 Budget Year', 'Ggrf Funding', 'Ggrf Budget Year',
       'Ctc Financial Resolution', 'Ctc  Amendment', 'Ctc Waiver',
       'Calsta Waiver', 'Date', 'Completion Date', 'Psa #', 'Ct Document #',
       '3Rd Party Award Date', 'Led', 'Date Branch Chief Receives Psa',
       'Date Regional Coordinator Receives Psa', 'Date Oc Receives Psa',
       'Date Opm Receives Psa', 'Date Legal Receives Psa',
       'Date Returned To Pm', 'Date Psa Sent To Local Agency',
       'Date Psa Approved By Local Agency', 'Date Signed By Drmt',
       'Psa Expiry Date', 'Lonp', 'Prior Fiscal Years To 2020',
       'Fiscal Year 2020 2021', 'Fiscal Year 2021 2022',
       'Fiscal Year 2022 2023', 'Fiscal Year 2023 2024',
       'Fiscal Year 2024 2025', 'Fiscal Year 2025 2026',
       'Fiscal Year 2026 2027', 'Fiscal Year 2027 2028',
      

In [32]:
# allocation.loc[allocation['allocation_award_year'] == 2020]

In [33]:
# project.loc[project['project_award_year'] == 2020]

In [34]:
project_cols = [
    "project_award_year",
    "project_project_#",
    "project_tircp_award_amount__$_",
    "project_grant_recipient",
    "project_project_title",
    "project_ppno",
    "project_unallocated_amount",
]

In [67]:
allocation_cols = [
    "allocation_award_year",
    "allocation_grant_recipient",
    "allocation_implementing_agency",
    "allocation_components",
    "allocation_ppno",
    "allocation_phase",
    "allocation_prior_fiscal_years_to_2020",
    "allocation_fiscal_year_2020_2021",
    "allocation_fiscal_year_2021_2022",
    "allocation_fiscal_year_2022_2023",
    "allocation_fiscal_year_2023_2024",
    "allocation_fiscal_year_2024_2025",
    "allocation_fiscal_year_2025_2026",
    "allocation_fiscal_year_2026_2027",
    "allocation_fiscal_year_2027_2028",
    "allocation_fiscal_year_2028_2029",
    "allocation_fiscal_year_2029_2030",
    "allocation_ctc_financial_resolution",
    "allocation_allocation_date",
    "allocation_project_id",
    "allocation_sb1_funding",
    "allocation_ggrf_funding",
    "allocation_allocation_amount",
]

In [68]:
missing_date = pd.to_datetime("2100-01-01")

In [69]:
group_by_cols = [
    "Project #",
    "Award No",
    "Award Amount",
    "Not Allocated",
    "Grant Recipient",
    "Implementing Agency",
    "Ppno",
    "Project Title",
    "Separable Phases/Components",
    "Phase",
    "Id",
    "Ctc Financial Resolution",
    "Allocation Date",
]

In [70]:
max_cols = [
    "Prior Fiscal Years To 2020",
    "Fiscal Year 2021 2022",
    "Fiscal Year 2022 2023",
    "Fiscal Year 2023 2024",
    "Fiscal Year 2024 2025",
    "Fiscal Year 2025 2026",
    "Fiscal Year 2026 2027",
    "Fiscal Year 2027 2028",
    "Fiscal Year 2028 2029",
    "Fiscal Year 2029 2030",
]

In [71]:
sum_cols = ["PTA-SB1_Amount", "Ggrf Funding", "Total Amount"]

In [72]:
monetary_cols = [max_cols + sum_cols] + ["Not Allocated", "Award Amount"]

In [73]:
def pivot(df):
    agg = df.groupby(group_by_cols).agg(
        {**{e: "max" for e in max_cols}, **{e: "sum" for e in sum_cols}}
    )
    return agg

In [79]:
def program_allocation_plan():

    # Load in Sheets
    df_project = A1_data_prep.clean_project()
    df_allocation = A1_data_prep.clean_allocation()

    # Only keeping certain columns
    df_project = df_project[project_cols]
    df_allocation = df_allocation[allocation_cols]

    # Merge
    m1 = df_allocation.merge(
        df_project,
        how="left",
        left_on=["allocation_ppno"],
        right_on=["project_ppno"],
    )

    # Clean Up
    # Delete one of the PPNO and Award Year Columns
    m1 = m1.drop(
        columns=[
            "allocation_award_year",
            "project_ppno",
            "allocation_grant_recipient",
        ]
    )

    # Fill in some columns with TBD so it'll show up
    m1[["allocation_project_id", "allocation_ctc_financial_resolution"]] = m1[
        ["allocation_project_id", "allocation_ctc_financial_resolution"]
    ].fillna(value="TBD")

    # Fill in missing dates with something random
    m1["allocation_allocation_date"] = m1["allocation_allocation_date"].fillna(
        missing_date
    )

    # Create Total_Amount Col
    m1["Total_Amount"] = m1["allocation_ggrf_funding"] + m1["allocation_sb1_funding"]

    # Create a column that concats award year + project #
    m1["Award No"] = (
        m1["project_award_year"].astype("str")
        + " : "
        + m1["project_project_#"].astype("str")
    )

    # Rename cols to the right names
    m1 = A1_data_prep.clean_up_columns(m1)
    m1 = m1.rename(
        columns={
            "Tircp Award Amount  $": "Award Amount",
            "#": "Project #",
            "Ctc Financial_Resolution": "Allocation Resolution",
            "Sb1 Funding": "PTA-SB1_Amount",
            "Unallocated Amount": "Not Allocated",
            "Title": "Project Title",
            "Components": "Separable Phases/Components",
            "Date": "Allocation Date",
        }
    )

    # Format to currency
    for i in monetary_cols:
        m1 = A1_data_prep.currency_format(m1, i)

    # Create sheets
    df_2015 = pivot(m1.loc[m1["Award Year"] == 2015])
    df_2016 = pivot(m1.loc[m1["Award Year"] == 2016])
    df_2018 = pivot(m1.loc[m1["Award Year"] == 2018])
    df_2020 = pivot(m1.loc[m1["Award Year"] == 2020])

    """
    # GCS
    with pd.ExcelWriter(f"{GCS_FILE_PATH}Program_Allocation_Plan.xlsx") as writer:
        df_2015.to_excel(writer, sheet_name="2015_Cycle_1", index=True)
        df_2016.to_excel(writer, sheet_name="2016_Cycle_2", index=True)
        df_2018.to_excel(writer, sheet_name="2018_Cycle_3", index=True)
        df_2020.to_excel(writer, sheet_name="2020_Cycle_4", index=True)
    """
    return m1

In [80]:
test = program_allocation_plan()

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed


In [82]:
test.head(1)

,Implementing Agency,Separable Phases/Components,Ppno,Phase,Prior Fiscal Years To 2020,Fiscal Year 2020 2021,Fiscal Year 2021 2022,Fiscal Year 2022 2023,Fiscal Year 2023 2024,Fiscal Year 2024 2025,Fiscal Year 2025 2026,Fiscal Year 2026 2027,Fiscal Year 2027 2028,Fiscal Year 2028 2029,Fiscal Year 2029 2030,Ctc Financial Resolution,Allocation Date,Id,PTA-SB1_Amount,Ggrf Funding,Amount,Award Year,Project #,Award Amount,Grant Recipient,Project Title,Not Allocated,Total Amount,Award No
0,Antelope Valley Transit Authority,Purchase 13 60-foot articulated BRT buses and 16 45-foot electric commuter buses,CP005,CONST,$24403000.0,0.0,$0.0,$0.0,$0.0,$0.0,$0.0,$0.0,$0.0,$0.0,$0.0,TIRCP-1516-02,2015-10-22,None,$0.0,$24403000.0,24403000.0,2015.0,1.0,$24403000.0,Antelope Valley Transit Authority,Regional Transit Interconnectivity & Environmental Sustability,$0.0,$24403000.0,2015.0 : 1.0
